In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
class NN:
    def __init__(self,X,Y):
        self.X = X
        self.Y = Y
    def classify(self):
        Xsize = self.X.shape[0]
        predict = []
        for test in self.X:
            inputTest = np.array([test])
            inputTest = np.repeat(inputTest,repeats=Xsize,axis=0)
            diff_input_X = self.X - inputTest
         
            diff_input_X = (diff_input_X ** 2).sum(axis=1)
            diff_input_X = (diff_input_X ** 0.5).argsort()
            votes = {1.0:0,2.0:0}
            
            if (self.X[diff_input_X[0]] == test).all():
                votes[self.Y[diff_input_X[1]]] += 1
            else:
                votes[self.Y[diff_input_X[0]]] += 1

            res = [0,0]
            for k,v in votes.items():
                if v > res[1]:
                    res[1] = v
                    res[0] = k

            predict.append(res[0])
        return predict
    
    def measure_accurancy(self,predicted):
        correct = 0      
        Ysize = self.Y.shape[0]
        for i in range(Ysize):       
            if predicted[i] == self.Y[i]:
                correct += 1
        return correct / Ysize
         
            
            
            
        
    


In [3]:
# forward selection & backward elimination
def forwardSelection_backwardElimination(X,Y,method):
    bestFeatures = []
    
    features = [i for i in range(X.shape[1])]
    if method == 'forward':
        feature_combinations = [[i] for i in features]
    else:
        feature_combinations = [[i for i in range(len(features))]]
    highest = [0,None]
    temp = []
    while True:
        f = feature_combinations.pop(0)
        nn = NN(X[:,f],Y)
        pred = nn.classify()
        res = nn.measure_accurancy(pred)
        temp.append([res,f])
        ans = [str(num+1) for num in f]
        print("    Using feature(s) {{{}}} accuracy is {}%".format(",".join([str(num+1) for num in f]),round(res*100,1)))
        
        if len(feature_combinations) == 0:
            temp.sort(key = lambda x : x[0], reverse=True)
            highest_pred = temp[0][0]
            if highest_pred > highest[0]:
                highest[0] = highest_pred
                highest[1] = temp[0][1]
            else:
                print("\n(Warning, Accuracy has decreased! Continuing search in case of local maxima)")
            print("\nFeature set {{{}}} was best, accuracy is {}%\n".
                  format(",".join([str(num+1) for num in temp[0][1]]),round(temp[0][0]*100,1)))
            bestFeatures.append([temp[0][1],round(temp[0][0]*100,1)])
            for i in temp:
                if i[0] == highest_pred:
                    if method == 'forward':
                        for j in features:
                            fes = [num for num in i[1]]
                            if j not in fes:
                                fes.append(j)
                                fes.sort()
                                if fes not in feature_combinations:
                                    feature_combinations.append(fes)
                    else:
                        for j in range(len(i[1])):
                            if len(i[1]) == 1:
                                continue
                            fes = [num for num in i[1]]
                            fes.pop(j)
                            fes.sort()
                            if fes not in feature_combinations:
                                feature_combinations.append(fes)

            temp = []
        if (method == 'forward' and len(f) == X.shape[1]) or (method == 'backward' and len(feature_combinations) == 0):
            break
       
    print("Finished search!! The best feature subset is {{{}}}, which has an accuracy of {}%".
          format(",".join([str(num+1) for num in highest[1]]),round(highest[0]*100,1)))
    return bestFeatures
        

In [4]:
#CS205_SP_2022_SMALLtestdata__29.txt
#CS205_SP_2022_Largetestdata__57.txt

print("Type the number of the algorithm you want to run.")
print("      1) Forward Selection")
print("      2) Backward Elimination")

while True:
    num = int(input())
    if 0 < num < 3:
        break
    else:
        print("Invalid number!!!")
        
while True:
    try:
        filename = input("Type in the name of the file to test : ")
        data = np.loadtxt(filename,dtype=float)
        X = data[:,1:]
        Y = data[:,:1].flatten()
        break
    except:
        print("Cannot find the file: {}".format(filename))


method = ['forward','backward']
print("\nThis dataset has {} features (not including the class attribute), with {} instances".format(X.shape[1],X.shape[0]))

nn = NN(X,Y)
pred = nn.classify()
res = nn.measure_accurancy(pred)
print("Running nearest neighbor with all {} features, using “leaving-one-out” evaluation, I get an\naccuracy of {}%\n"
      .format(X.shape[1],round(res*100,1)))

print("Beginning  search")
A = np.count_nonzero(Y == 1.0)
B = np.count_nonzero(Y == 2.0)
default = 0
if A > B:
    default = round((A / Y.shape[0])*100,1)
else:
    default = round((B / Y.shape[0])*100,1)

s = time.time()
if method == 'forward':
    resFeatures = [[[],default]]
    resFeatures += forwardSelection_backwardElimination(X,Y,method[num-1])
else:
    resFeatures = forwardSelection_backwardElimination(X,Y,method[num-1])
    resFeatures.append([[],default])

end = time.time()
print("Total time spend:{}s".format(round(end - s,2)))

Type the number of the algorithm you want to run.
      1) Forward Selection
      2) Backward Elimination
1
Type in the name of the file to test : CS205_SP_2022_SMALLtestdata__29.txt

This dataset has 10 features (not including the class attribute), with 300 instances
Running nearest neighbor with all 10 features, using “leaving-one-out” evaluation, I get an
accuracy of 76.3%

Beginning  search
    Using feature(s) {1} accuracy is 71.7%
    Using feature(s) {2} accuracy is 78.3%
    Using feature(s) {3} accuracy is 70.0%
    Using feature(s) {4} accuracy is 73.7%
    Using feature(s) {5} accuracy is 71.3%
    Using feature(s) {6} accuracy is 79.7%
    Using feature(s) {7} accuracy is 70.3%
    Using feature(s) {8} accuracy is 69.0%
    Using feature(s) {9} accuracy is 72.7%
    Using feature(s) {10} accuracy is 73.7%

Feature set {6} was best, accuracy is 79.7%

    Using feature(s) {1,6} accuracy is 74.7%
    Using feature(s) {2,6} accuracy is 94.0%
    Using feature(s) {3,6} accurac